In [4]:
## 00.data_scraping.ipynbのスクレイピングデータを抽出、加工する。馬ごとの近5走における各特徴量データを追加。

import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

def class_mapping(row):
    # クラス名を数値にマッピング
    mappings = {'障害': 0, 'G1': 10, 'G2': 9, 'G3': 8, '(L)': 7, 'オープン': 7, 'OP': 7, '3勝': 6, '1600': 6, '2勝': 5, '1000': 5, '1勝': 4, '500': 4, '新馬': 3, '未勝利': 1}
    for key, value in mappings.items():
        if key in row:
            return value
    return 0

def standardize_times(df, col_name):
    # 走破時間を秒に変換
    time_parts = df[col_name].str.split(':', expand=True)
    seconds = time_parts[0].astype(float) * 60 + time_parts[1].str.split('.', expand=True)[0].astype(float) + time_parts[1].str.split('.', expand=True)[1].astype(float) / 10
    seconds = seconds.ffill()
    
    # 平均と標準偏差を計算し標準化
    mean_seconds = seconds.mean()
    std_seconds = seconds.std()
    df[col_name] = -((seconds - mean_seconds) / std_seconds)
    
    # 外れ値処理
    df[col_name] = df[col_name].apply(lambda x: -3 if x < -3 else (2 if x > 2.5 else x))
    
    # 再度標準化
    mean_seconds_2 = df[col_name].mean()
    std_seconds_2 = df[col_name].std()
    df[col_name] = (df[col_name] - mean_seconds_2) / std_seconds_2
    
    return mean_seconds, std_seconds, mean_seconds_2, std_seconds_2

def add_seasonal_features(df, date_columns):
    # 日付カラムから季節特徴量を追加
    for date_col in date_columns:
        if not np.issubdtype(df[date_col].dtype, np.datetime64):
            df[date_col] = pd.to_datetime(df[date_col])
        df[f'{date_col}_sin'] = np.sin((df[date_col].dt.month - 1) * (2 * np.pi / 12))
        df[f'{date_col}_cos'] = np.cos((df[date_col].dt.month - 1) * (2 * np.pi / 12))

yearStart = 2022
yearEnd = 2023
yearList = np.arange(yearStart, yearEnd + 1)
dfs = []

print("ファイル取得：開始")

for year in yearList:
    file_path = f"data/{year}.csv"
    df = pd.read_csv(file_path, encoding="SHIFT-JIS", header=0)
    df['日付'] = pd.to_datetime(df['日付'], format='%Y年%m月%d日', errors='coerce')
    df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
    df = df.dropna(subset=['着順'])
    df['着順'] = df['着順'].astype(int)
    df['賞金'] = pd.to_numeric(df['賞金'], errors='coerce').fillna(0)
    dfs.append(df)

df_combined = pd.concat(dfs, ignore_index=True)

print("ファイル取得：完了")
print("データ変換：開始")

# NaNが含まれる行を削除
df_combined = df_combined.dropna(subset=['走破時間'])

# 走破時間の標準化
mean_seconds, std_seconds, mean_seconds_2, std_seconds_2 = standardize_times(df_combined, '走破時間')

print('1回目平均' + str(mean_seconds))
print('2回目平均' + str(mean_seconds_2))
print('1回目標準偏差' + str(std_seconds))
print('2回目標準偏差' + str(std_seconds_2))

# 標準化情報をCSVに保存
time_df = pd.DataFrame({
    'Mean': [mean_seconds, mean_seconds_2],
    'Standard Deviation': [std_seconds, std_seconds_2]
}, index=['First Time', 'Second Time'])
time_df.to_csv('config/standard_deviation.csv')

# 通過順の平均を計算
pas = df_combined['通過順'].str.split('-', expand=True)
df_combined['通過順'] = pas.astype(float).mean(axis=1)

# マッピング情報を適用
mappings = {
    '性': {'牡': 0, '牝': 1, 'セ': 2},
    '芝・ダート': {'芝': 0, 'ダ': 1, '障': 2},
    '回り': {'右': 0, '左': 1, '芝': 2, '直': 2},
    '馬場': {'良': 0, '稍': 1, '重': 2, '不': 3},
    '天気': {'晴': 0, '曇': 1, '小': 2, '雨': 3, '雪': 4}
}
for column, mapping in mappings.items():
    df_combined[column] = df_combined[column].map(mapping)

# クラス変換を適用
df_combined['クラス'] = df_combined['クラス'].apply(class_mapping)

print("データ変換：完了")
print("近5走取得：開始")

# データをソート
df_combined.sort_values(by=['馬', '日付'], ascending=[True, False], inplace=True)

features = ['馬番', '騎手', '斤量', 'オッズ', '体重', '体重変化', '上がり', '通過順', '着順', '距離', 'クラス', '走破時間', '芝・ダート', '天気', '馬場']

# 各馬の過去5走の情報をシフトしながら取得し、ffillで欠損値を補完
shifts = {}
for i in range(1, 6):
    shifts[f'日付{i}'] = df_combined.groupby('馬')['日付'].shift(-i)
    for feature in features:
        shifts[f'{feature}{i}'] = df_combined.groupby('馬')[feature].shift(-i).ffill()

# 新しい列を一度にDataFrameに追加
df_combined = pd.concat([df_combined, pd.DataFrame(shifts)], axis=1)

# race_idと馬でグルーピングし、最新の特徴量を取得
df_combined = df_combined.groupby(['race_id', '馬'], as_index=False).last()
df_combined.sort_values(by='race_id', ascending=False, inplace=True)

print("近5走取得：終了")
print("日付変換：開始")

df_combined.replace('---', np.nan, inplace=True)

# 距離差と日付差を計算
df_combined['距離差'] = df_combined['距離'] - df_combined['距離1']
df_combined['日付差'] = (df_combined['日付'] - df_combined['日付1']).dt.days
for i in range(1, 5):
    df_combined[f'距離差{i}'] = df_combined[f'距離{i}'] - df_combined[f'距離{i+1}']
    df_combined[f'日付差{i}'] = (df_combined[f'日付{i}'] - df_combined[f'日付{i+1}']).dt.days

# 斤量関連の列を数値に変換し、変換できないデータはNaNに
kinryo_columns = ['斤量', '斤量1', '斤量2', '斤量3', '斤量4', '斤量5']
df_combined[kinryo_columns] = df_combined[kinryo_columns].apply(pd.to_numeric, errors='coerce')
df_combined['平均斤量'] = df_combined[kinryo_columns].mean(axis=1)

# 騎手の勝率を計算し、保存およびマージ
jockey_win_rate = df_combined.groupby('騎手')['着順'].apply(lambda x: (x == 1).sum() / x.count()).reset_index()
jockey_win_rate.columns = ['騎手', '騎手の勝率']
jockey_win_rate.to_csv('calc_rate/jockey_win_rate.csv', index=False)
df_combined = pd.merge(df_combined, jockey_win_rate, on='騎手', how='left')

# 各レースの出走頭数を計算
df_combined['出走頭数'] = df_combined.groupby('race_id')['race_id'].transform('count')

# 各馬に対して過去5レースの出走頭数を特徴量として追加
for i in range(1, 6):
    df_combined[f'出走頭数{i}'] = df_combined.groupby('馬')['出走頭数'].shift(i).fillna(0)

# 距離と走破時間からスピードを計算し、平均スピードを新しい列として追加
for i in range(1, 6):
    df_combined[f'スピード{i}'] = df_combined[f'距離{i}'] / df_combined[f'走破時間{i}']
df_combined['平均スピード'] = df_combined[[f'スピード{i}' for i in range(1, 6)]].mean(axis=1, skipna=True)
df_combined.drop(columns=[f'スピード{i}' for i in range(1, 6)], inplace=True)

# 過去5走の賞金を取得し、賞金合計を計算
for i in range(1, 6):
    df_combined[f'賞金{i}'] = df_combined.groupby('馬')['賞金'].shift(i)
df_combined['過去5走の合計賞金'] = df_combined[[f'賞金{i}' for i in range(1, 6)]].sum(axis=1)
df_combined.drop(columns=[f'賞金{i}' for i in range(1, 6)] + ['賞金'], inplace=True)

df_combined.sort_values(by='race_id', ascending=False, inplace=True)

# 日付カラムから年、月、日を抽出
df_combined['year'] = df_combined['日付'].dt.year
df_combined['month'] = df_combined['日付'].dt.month
df_combined['day'] = df_combined['日付'].dt.day

# 季節特徴量を追加
date_columns = ['日付1', '日付2', '日付3', '日付4', '日付5']
add_seasonal_features(df_combined, date_columns)

date_columns = ['日付', '日付1', '日付2', '日付3', '日付4', '日付5']
for col in date_columns:
    df_combined['year'] = df_combined[col].dt.year
    df_combined['month'] = df_combined[col].dt.month
    df_combined['day'] = df_combined[col].dt.day
    df_combined[col] = (df_combined['year'] - yearStart) * 365 + df_combined['month'] * 30 + df_combined['day']
df_combined.drop(['year', 'month', 'day'], axis=1, inplace=True)

print("日付変換：終了")

# 騎手の乗り替わり特徴量を追加
df_combined['騎手の乗り替わり'] = df_combined.groupby('馬')['騎手'].transform(lambda x: (x != x.shift()).astype(int))

# カテゴリカル変数のラベルエンコーディング
categorical_features = ['馬', '騎手', '調教師', 'レース名', '開催', '場名', '騎手1', '騎手2', '騎手3', '騎手4', '騎手5']
for i, feature in enumerate(categorical_features):
    print(f"\rProcessing feature {i+1}/{len(categorical_features)}", end="")
    le = LabelEncoder()
    df_combined[feature] = le.fit_transform(df_combined[feature].astype(str))

# エンコーディングとスケーリング後のデータを確認
print("ファイル出力：開始")
df_combined.to_csv(f'encoded/{yearStart}_{yearEnd}encoded_data.csv', index=False)
print("ファイル出力：終了")



# # データの読み込み
# data = pd.read_csv('encoded/encoded_data.csv')

# # 特徴量エンジニアリングの追加 ----------------------------------------
# # 騎手と調教師の組み合わせ（列が存在する場合）
# if '騎手' in data.columns and '調教師' in data.columns:
#     data['騎手_調教師'] = data['騎手'] + '_' + data['調教師']

# # 過去3走の平均順位（馬の履歴データがある場合）
# if '着順' in data.columns and '馬' in data.columns:
#     data['過去3走平均順位'] = data.groupby('馬')['着順'].transform(
#         lambda x: x.rolling(3, min_periods=1).mean()  # 最低1走分あれば計算
#     )
# # ------------------------------------------------------------

# # 着順を変換
# data['着順'] = data['着順'].map(lambda x: 1 if x < 4 else 0)

ファイル取得：開始
ファイル取得：完了
データ変換：開始
1回目平均102.60497154002307
2回目平均0.015193978769472035
1回目標準偏差29.22003906359807
2回目標準偏差0.9422848306121085
データ変換：完了
近5走取得：開始
近5走取得：終了
日付変換：開始
日付変換：終了
Processing feature 11/11ファイル出力：開始
ファイル出力：終了
